In [1]:
# from langchain_community.tools import DuckDuckGoSearchRun
# search=DuckDuckGoSearchRun()
# search.run("latest news about Mars rover")

In [19]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [20]:
api_wrapper= WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [21]:
wiki.name

'wikipedia'

In [22]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader=WebBaseLoader("https://medium.com/%40Shamimw/understanding-langchain-tools-and-agents-a-guide-to-building-smart-ai-applications-e81d200b3c12")
doc=loader.load()
documents= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50).split_documents(doc)
db= FAISS.from_documents(documents, HuggingFaceEmbeddings())
retriever=db.as_retriever()

C:\Users\pthorat\AppData\Local\Temp\ipykernel_21536\2025247378.py:8: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  db= FAISS.from_documents(documents, HuggingFaceEmbeddings())


In [5]:
##creating retriever tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool= create_retriever_tool(retriever, "langchain_tools",
                      "search information about diffrent tools and agents. for any question related with tools and agent you must use this, Input should be a string."
                )

In [6]:
from langchain.tools import Tool

# Wrap retriever in a str-friendly Tool
retriever_tool_wrapped = Tool(
    name="langchain_tools",
    func=lambda x: retriever_tool.run({"query":x}),
    description="Search information about different tools and agents. Input should be a string."
)


In [7]:
result=retriever_tool_wrapped.run("what is types of tool")
result

'Tools — Enable interaction with external APIs (e.g., weather data, stock prices, etc.).Mathematical Tools — Allow AI to perform complex calculations and data processing.File Processing Tools — Help analyze documents, PDFs, or CSV files.Code Execution Tools — Provide AI the capability to write, execute, and debug code in Python or other languages.Example: Using Tools in LangChainHere’s a simple example of defining a LangChain tool that performs a mathematical calculation:from langchain.tools\n\nthat allow an AI model (such as GPT-4) to interact with external systems, retrieve data, or perform actions beyond simple text generation. These tools act as APIs or function calls that the AI can invoke when needed.Types of LangChain ToolsSearch Tools — Allow models to fetch real-time information from sources like Google Search or Bing.Database Query Tools — Help AI interact with structured data sources like SQL databases or vector stores.API Call Tools — Enable interaction with external APIs\n

In [17]:
from langchain.agents import Tool, initialize_agent, AgentType
import requests


def get_npi_info(npi):
    url = f"https://npiregistry.cms.hhs.gov/api/?number={npi}&version=2.1"
    responce= requests.get(url)
    data=responce.json()
    # return data
    if responce.status_code!=200:
        return("Failed to get result")
    else:
        result = data["results"][0]
        org_name = result["basic"]["organization_name"]
        address = result["addresses"][0]
        address_info = {
            "address_1": address.get("address_1", ""),
            "city": address.get("city", ""),
            "state": address.get("state", ""),
            "postal_code": address.get("postal_code", "")
        }

    return f"Organization name for NPI {npi}: {org_name} with address {address_info}"

npi_tool = Tool(
    name= "npi_info_tool",
    func= get_npi_info,
    description= "use this to get organization name or hospital name of given npi numbers like 1972503464 with address."
)

In [9]:
npi_tool.name

'npi_info_tool'

In [10]:
tools= [wiki, retriever_tool_wrapped, npi_tool]

In [11]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\pthorat\\Desktop\\Genai\\myenv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langchain_tools', description='Search information about different tools and agents. Input should be a string.', func=<function <lambda> at 0x0000013DC7255BC0>),
 Tool(name='npi_info_tool', description='use this to get organization name or hospital name of given npi numbers like 1972503464 with address.', func=<function get_npi_info at 0x0000013DC7255580>)]

In [12]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

c:\Users\pthorat\Desktop\Genai\myenv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [13]:
from langchain_community.chat_models import ChatOpenAI
# from langchain_community.llms import Ollama
from dotenv import load_dotenv
load_dotenv()
llm= ChatOpenAI(model='gpt-4o-mini', max_tokens=200)
# llm= Ollama(model="llama2")

C:\Users\pthorat\AppData\Local\Temp\ipykernel_21536\699189938.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm= ChatOpenAI(model='gpt-4o-mini', max_tokens=200)


In [14]:
agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

C:\Users\pthorat\AppData\Local\Temp\ipykernel_21536\4179209173.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [16]:
agent_executor.run("Tell me about tools in langchain")



> Entering new AgentExecutor chain...
I need to gather information about the tools available in Langchain. 
Action: langchain_tools
Action Input: "tools in langchain"
Observation: Understanding LangChain Tools and Agents: A Guide to Building Smart AI Applications | by WS | MediumSitemapOpen in appSign upSign inMedium LogoWriteSign upSign inUnderstanding LangChain Tools and Agents: A Guide to Building Smart AI ApplicationsWSFollow9 min read·Feb 16, 2025--ListenShareThe rise of AI-powered applications has brought significant advancements in natural language processing (NLP) and automation. LangChain, an open-source framework, has emerged as a powerful tool for developing

has emerged as a powerful tool for developing applications that integrate language models with external tools, knowledge bases, and APIs. At the core of LangChain’s functionality are Tools and Agents, which enable AI models to perform actions dynamically.This article explores LangChain’s Tools and Agents, how they wor

'LangChain tools are interfaces that allow AI models, such as GPT-4, to interact with external systems, retrieve data, or perform actions beyond simple text generation. They can be categorized into several types:\n\n1. **Search Tools**: Enable models to fetch real-time information from sources like Google Search or Bing.\n2. **Database Query Tools**: Allow interaction with structured data sources like SQL databases or vector stores.\n3. **API Call Tools**: Facilitate interaction with external APIs for various data retrieval and actions.\n4. **Mathematical Tools**: Enable complex calculations and data processing.\n5. **File Processing Tools**: Assist in analyzing documents, PDFs, or CSV files.\n6. **Code Execution Tools**: Allow writing, executing, and debugging code in programming languages like Python.\n\nThese tools enhance the capabilities of AI applications by enabling them to perform dynamic actions based'

In [ ]:
# from langchain_community.llms import Ollama
# llm= Ollama(model="llama2")

# agent = initialize_agent(
#     tools=tools,
#     llm=llm,
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     verbose=True
# )